In [275]:
# Import basic libraries
import numpy as np
import pandas as pd

In [276]:
# import both the datasets for this 
movies_data = pd.read_csv("../Datasets/Prj-MovieRecommender/tmdb_5000_movies.csv")
credits = pd.read_csv("../Datasets/Prj-MovieRecommender/tmdb_5000_credits.csv")

In [277]:
movies_data.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [278]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [279]:
# try reading 'genres' to find names for it...A test
import json
recs = movies_data.head(2)
genre = json.loads(recs['genres'][0])
for gen in genre:
    print(gen['name'])

Action
Adventure
Fantasy
Science Fiction


In [280]:
# try reading 'casts' to find characters and artist for it
# ...A test
casts = credits.head(1)['cast'][0]
cast = json.loads(casts)
for ca in cast:
    print('Character  : ', ca['character'])
    print('Played by  : ', ca['name'], '\n')

Character  :  Jake Sully
Played by  :  Sam Worthington 

Character  :  Neytiri
Played by  :  Zoe Saldana 

Character  :  Dr. Grace Augustine
Played by  :  Sigourney Weaver 

Character  :  Col. Quaritch
Played by  :  Stephen Lang 

Character  :  Trudy Chacon
Played by  :  Michelle Rodriguez 

Character  :  Selfridge
Played by  :  Giovanni Ribisi 

Character  :  Norm Spellman
Played by  :  Joel David Moore 

Character  :  Moat
Played by  :  CCH Pounder 

Character  :  Eytukan
Played by  :  Wes Studi 

Character  :  Tsu'Tey
Played by  :  Laz Alonso 

Character  :  Dr. Max Patel
Played by  :  Dileep Rao 

Character  :  Lyle Wainfleet
Played by  :  Matt Gerald 

Character  :  Private Fike
Played by  :  Sean Anthony Moran 

Character  :  Cryo Vault Med Tech
Played by  :  Jason Whyte 

Character  :  Venture Star Crew Chief
Played by  :  Scott Lawrence 

Character  :  Lock Up Trooper
Played by  :  Kelly Kilgour 

Character  :  Shuttle Pilot
Played by  :  James Patrick Pitt 

Character  :  Shut

In [281]:
# try reading 'crew' to find department, job and the name of the person doing the job
# ...A test
crews = credits.head(1)['crew'][0]
crew = json.loads(crews)
for ca in crew:
    print('\nDepartment :', ca['department'] )
    print( 'Job        :', ca['job'])
    print( 'Name       :', ca['name'])


Department : Editing
Job        : Editor
Name       : Stephen E. Rivkin

Department : Art
Job        : Production Design
Name       : Rick Carter

Department : Sound
Job        : Sound Designer
Name       : Christopher Boyes

Department : Sound
Job        : Supervising Sound Editor
Name       : Christopher Boyes

Department : Production
Job        : Casting
Name       : Mali Finn

Department : Sound
Job        : Original Music Composer
Name       : James Horner

Department : Directing
Job        : Director
Name       : James Cameron

Department : Writing
Job        : Writer
Name       : James Cameron

Department : Editing
Job        : Editor
Name       : James Cameron

Department : Production
Job        : Producer
Name       : James Cameron

Department : Writing
Job        : Screenplay
Name       : James Cameron

Department : Art
Job        : Art Direction
Name       : Andrew Menzies

Department : Visual Effects
Job        : Visual Effects Producer
Name       : Jill Brooks

Department

In [282]:
# check if titles are non-unique. Will use it to merge if they are unique
sum(credits.title.value_counts() > 1)
# oops...!  'title' has duplicacy. Hence I will not use it to merge two data-frames
# A better approiach will be to drop duplicate records. Records having same title

3

In [283]:
# check those cases where 'title' in movies_data is not equal to 'title' in credits
movies_data['title'].value_counts()

The Host                 2
Batman                   2
Out of the Blue          2
Avatar                   1
The Girl on the Train    1
                        ..
Step Up 3D               1
Secondhand Lions         1
The Age of Adaline       1
Drag Me to Hell          1
My Date with Drew        1
Name: title, Length: 4800, dtype: int64

In [284]:
credits['title'].value_counts()

The Host                 2
Batman                   2
Out of the Blue          2
Avatar                   1
The Girl on the Train    1
                        ..
Step Up 3D               1
Secondhand Lions         1
The Age of Adaline       1
Drag Me to Hell          1
My Date with Drew        1
Name: title, Length: 4800, dtype: int64

In [285]:
# Merge two dataframes based on the 'id' and 'movie_id'
df = movies_data.merge(credits, left_on='id', right_on='movie_id')
#df = movies.merge(credits, on='title')
df.shape

# looks good as number of records are not increased or decreased. 
# 'id' and 'movie_id' are same value but I will deal with it during feature selection

(4803, 24)

In [286]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title_x', 'vote_average',
       'vote_count', 'movie_id', 'title_y', 'cast', 'crew'],
      dtype='object')

In [287]:
# drom 'movie_id' as it is same as 'id'
# drop 'title_y' column and chnage name of the 'title_x' to just 'title'
movies = df.drop(['movie_id', 'title_y'], axis=1)
movies = movies.rename(columns={'title_x': 'title'})
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'cast', 'crew'],
      dtype='object')

In [288]:
# drop the duplicate records (although they have unique 'id' but everytrhing else is same.)
movies = movies.drop_duplicates(subset=['title'])
movies.shape

(4800, 22)

In [289]:
# check total columns in the data-frame
print("Columns : ", movies.columns)
movies.info()

Columns :  Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'cast', 'crew'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4800 entries, 0 to 4802
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4800 non-null   int64  
 1   genres                4800 non-null   object 
 2   homepage              1711 non-null   object 
 3   id                    4800 non-null   int64  
 4   keywords              4800 non-null   object 
 5   original_language     4800 non-null   object 
 6   original_title        4800 non-null   object 
 7   overview              4797 non-null   object 
 8   popularity  

In [290]:
# status of the movie is mostly release. Highly imbalances dataset
# hence will ignore this columns
movies.status.value_counts()

Released           4792
Rumored               5
Post Production       3
Name: status, dtype: int64

<!-- ## Here we are skipping numerical values columns as NLP and text based algorithms are considered
##
##

# columns to keep (these column seems to make sense in decision making)
#
# ['genres'                 : important as it tells the flavour and theme
# 'id' (Ignore-Later)       : Not useful in model training. Will be meaningful in identifying related information later
# 'keywords'                : Tag for the movie. Very important information
# 'overview'                : Will define plot and description keywords. Will be helpful.
# 'title'                   : May not be important but let us keep it.
# 'cast'                    : Very important 
# 'crew'                    : Very imporatant
# ]
#

# Columns to drop as they do not appear to e useful in our deecision making
#
# 'budget' (Later)           : Should be useful but for now ignore it. Budget means resources used in creating a movie. More resources more mature graphics, locations, actors fee, set quality etc
# 'original_language'        : Most of the movies are in english. So let us drop it as it does not contribute to the variability in the dataset
#                              movies[].value_counts() 
# 'original_title'           : it may be in non-english hence cause issues. Will not add much value.
# 'popularity' (Later)       : people do watch mivies by popularity. However it can be ignored in our problem solving approach.
# 'production_companies' (Later): Should be useful but ignore for now
# 'production_countries' (Later): We do care for countries of origin. But not the shooting places. It's ambiguous.
# 'release_date' (Later)     : Time of release does matter
# 'revenue' (Later)          : Revenue earned may be indicator of how many people liked it. Despite individuals choices.
#                              Though it may be meaningless if we want to cater specific class of people. Mobocracy may not work :-) 
# 'runtime'                  : Duration should not matter   
# 'spoken_languages'         : Should not matter
# 'status'                   : status is mostly 'release'. Highly imbalances dataset, hence will ignore this columns.
# 'tagline'                  : tagline does not matte. It may be totally irrelevant. Does not convey reson to cause of liking
# 'vote_average'   (Later)   : Valuabe but numeric       
# 'vote_count'   (Later)     : Valuabe but numeric. will re-evaluate as common people tend to rate very-good and very-bad movies only    
# 'homepage'] -->

In [291]:
## Here we are skipping numerical values columns as NLP and text based algorithms are considered
##
##

# columns to keep (these column seems to make sense in decision making)
#
# ['genres'                 : important as it tells the flavour and theme
# 'id' (Ignore-Later)       : Not useful in model training. Will be meaningful in identifying related information later
# 'keywords'                : Tag for the movie. Very important information
# 'overview'                : Will define plot and description keywords. Will be helpful.
# 'title'                   : May not be important but let us keep it.
# 'cast'                    : Very important 
# 'crew'                    : Very imporatant
# ]
#

# Columns to drop as they do not appear to e useful in our deecision making
#
# 'budget' (Later)           : Should be useful but for now ignore it. Budget means resources used in creating a movie. More resources more mature graphics, locations, actors fee, set quality etc
# 'original_language'        : Most of the movies are in english. So let us drop it as it does not contribute to the variability in the dataset
#                              movies[].value_counts() 
# 'original_title'           : it may be in non-english hence cause issues. Will not add much value.
# 'popularity' (Later)       : people do watch mivies by popularity. However it can be ignored in our problem solving approach.
# 'production_companies' (Later): Should be useful but ignore for now
# 'production_countries' (Later): We do care for countries of origin. But not the shooting places. It's ambiguous.
# 'release_date' (Later)     : Time of release does matter
# 'revenue' (Later)          : Revenue earned may be indicator of how many people liked it. Despite individuals choices.
#                              Though it may be meaningless if we want to cater specific class of people. Mobocracy may not work :-) 
# 'runtime'                  : Duration should not matter   
# 'spoken_languages'         : Should not matter
# 'status'                   : status is mostly 'release'. Highly imbalances dataset, hence will ignore this columns.
# 'tagline'                  : tagline does not matte. It may be totally irrelevant. Does not convey reson to cause of liking
# 'vote_average'   (Later)   : Valuabe but numeric       
# 'vote_count'   (Later)     : Valuabe but numeric. will re-evaluate as common people tend to rate very-good and very-bad movies only    
# 'homepage']

In [292]:
# keep the columns of interest only
movies = movies[['id', 'title', 'overview', 'genres', 'keywords',  'cast', 'crew']]

In [295]:
movies.sample(2)

,id,title,overview,genres,keywords,cast,crew
3466,10215,Sliding Doors,"Gwyneth Paltrow plays London publicist Helen, ...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 848, ""name"": ""double life""}, {""id"": 10...","[{""cast_id"": 1, ""character"": ""Helen Quilley"", ...","[{""credit_id"": ""5932d476c3a368599000d5dd"", ""de..."
1481,202575,Thunder and the House of Magic,"Thunder, an abandoned young cat seeking shelte...","[{""id"": 10751, ""name"": ""Family""}, {""id"": 14, ""...","[{""id"": 2343, ""name"": ""magic""}, {""id"": 209714,...","[{""cast_id"": 26, ""character"": ""Thunder / Dylan...","[{""credit_id"": ""53820c430e0a2624c700a682"", ""de..."
